In [40]:
%tensorflow_version 2.x #this  line is not required unless you are in a notebook
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np  #opt array lib dot, cross, etc
import pandas as pd #data analytics tool, manipulate data, visualize data
import matplotlib.pyplot as plt # visualization tool graphs, plots etc
from IPython.display import clear_output # clear the output for this notebok
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.15.0


In [41]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
#print(dftrain.head())
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
#print(dftrain.head())
print(dftrain.loc[0], y_train.loc[0])

sex                          male
age                          22.0
n_siblings_spouses              1
parch                           0
fare                         7.25
class                       Third
deck                      unknown
embark_town           Southampton
alone                           n
Name: 0, dtype: object 0


In [80]:
dftrain.describe()



{'sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male', 5: 'male', 6: 'female', 7: 'female', 8: 'female', 9: 'male', 10: 'male', 11: 'female', 12: 'male', 13: 'male', 14: 'female', 15: 'female', 16: 'male', 17: 'male', 18: 'female', 19: 'male', 20: 'male', 21: 'female', 22: 'male', 23: 'male', 24: 'female', 25: 'female', 26: 'male', 27: 'male', 28: 'male', 29: 'male', 30: 'female', 31: 'female', 32: 'female', 33: 'male', 34: 'female', 35: 'male', 36: 'male', 37: 'male', 38: 'female', 39: 'male', 40: 'female', 41: 'female', 42: 'male', 43: 'male', 44: 'female', 45: 'male', 46: 'male', 47: 'male', 48: 'female', 49: 'male', 50: 'male', 51: 'male', 52: 'male', 53: 'female', 54: 'male', 55: 'male', 56: 'male', 57: 'male', 58: 'male', 59: 'male', 60: 'male', 61: 'male', 62: 'female', 63: 'male', 64: 'male', 65: 'female', 66: 'male', 67: 'male', 68: 'male', 69: 'female', 70: 'male', 71: 'male', 72: 'male', 73: 'male', 74: 'male', 75: 'male', 76: 'female', 77: 'male', 78: 'female

In [107]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
#print(dftrain.loc[0])
dftrain_dict = dftrain.to_dict('list')
#print(dftrain_dict)

# Plain float values.
tf.keras.utils.FeatureSpace.float(name=None)

# Float values to be preprocessed via featurewise standardization
# (i.e. via a `keras.layers.Normalization` layer).
tf.keras.utils.FeatureSpace.float_normalized(name=None)

# Float values to be preprocessed via linear rescaling
# (i.e. via a `keras.layers.Rescaling` layer).
tf.keras.utils.FeatureSpace.float_rescaled(scale=1., offset=0., name=None)


# Integer values to be indexed. By default, the discrete
# representation will then be one-hot encoded.
tf.keras.utils.FeatureSpace.integer_categorical(
    max_tokens=None, num_oov_indices=1, output_mode="one_hot", name=None)

# String values to be indexed. By default, the discrete
# representation will then be one-hot encoded.
tf.keras.utils.FeatureSpace.string_categorical(
    max_tokens=None, num_oov_indices=1, output_mode="one_hot", name=None)



feature_space = tf.keras.utils.FeatureSpace(
    features={
        "sex": "string_categorical",
        "n_siblings_spouses": "integer_categorical",
        "parch": "integer_categorical",
        "class": "string_categorical",
        "deck": "string_categorical",
        "embark_town": "string_categorical",
        "alone": "string_categorical",
        "age": "float_normalized",
        "fare": "float_normalized",

    },
    output_mode="concat",
)

dataset = tf.data.Dataset.from_tensor_slices(dftrain_dict)
feature_space.adapt(dataset)

feature_columns = feature_space(dftrain_dict)

print(feature_columns[0])
feature_columns_1 = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns_1.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns_1.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
  #feature_columns.append(tf.keras.utils.FeatureSpace(features={feature_name: "float_normalized"}))

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((data_df.to_dict(), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use
dftrain.loc[0]
train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

inear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result=linear_est.evaluate(eval_input_fn)



tf.Tensor(
[-0.610415   0.         0.         1.         0.         1.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  1.         0.         0.         0.        -0.4974028  0.
  0.         1.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.
  0.         0.         0.         1.         0.       ], shape=(41,), dtype=float32)


ValueError: Unbatching a tensor is only supported for rank >= 1

In [110]:
print(feature_columns[3])
print(feature_columns_1[3])

import tensorflow as tf

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

# Assume dftrain_dict is a dictionary containing your dataset

# Define input layers for categorical columns
input_layers_cat = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    input_layer_cat = tf.keras.layers.Input(shape=(1,), name=feature_name, dtype=tf.string)
    input_layers_cat.append(input_layer_cat)

# Embedding layers for categorical columns
embedding_layers = []
for input_layer_cat in input_layers_cat:
    embedding_layers.append(tf.keras.layers.Embedding(input_dim=len(vocabulary), output_dim=10)(input_layer_cat))

# Input layers for numeric columns
input_layers_num = []
for feature_name in NUMERIC_COLUMNS:
    input_layer_num = tf.keras.layers.Input(shape=(1,), name=feature_name, dtype=tf.float32)
    input_layers_num.append(input_layer_num)

# Concatenate all input layers
all_input_layers = input_layers_cat + input_layers_num

# Merge all feature columns
feature_columns = tf.keras.layers.concatenate(embedding_layers + input_layers_num)

# Your model architecture goes here
# ...

# Compile and train the model
# ...

# Example usage in your input function
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        input_dict = {col: data_df[col].values for col in CATEGORICAL_COLUMNS + NUMERIC_COLUMNS}
        ds = tf.data.Dataset.from_tensor_slices((input_dict, label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

# Train and evaluate your model
# ...


tf.Tensor(
[0.42943236 0.         0.         1.         0.         0.
 1.         0.         0.         0.         1.         0.
 0.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.34304613 0.
 0.         1.         0.         0.         0.         0.
 0.         0.         1.         0.         0.         0.
 0.         0.         0.         0.         1.        ], shape=(41,), dtype=float32)
VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 1, 9), (None, 1, 9), (None, 1, 9), (None, 1, 9), (None, 1, 9), (None, 1, 9), (None, 1, 9), (None, 1), (None, 1)]

In [100]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((data_df.to_dict(), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use
dftrain.loc[0]
train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [101]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result=linear_est.evaluate(eval_input_fn)
clear_output()
print(result["accuracy"])
print(result)

ValueError: Unbatching a tensor is only supported for rank >= 1

In [47]:
print(dfeval.shape)
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[3])
print(y_eval.loc[3])
print(result[3]['probabilities'][1])

(264, 9)
sex                        female
age                          55.0
n_siblings_spouses              0
parch                           0
fare                         16.0
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 3, dtype: object
1
0.59879315
